In [91]:
import pandas as pd
pd.set_option('max_colwidth', 100)

siads_csv = pd.read_csv('2024-10-17-SIADS_RelatorioInventario.csv', on_bad_lines='skip', encoding='utf-8')

# siads_csv = siads_csv[['Unnamed: 0', 'Unnamed: 12', 'Unnamed: 14']]
uasg = '151911'
ug = '26418'
ourg = '74691'
# Remover valores NaN da coluna 'Unnamed: 0' antes de aplicar .str.contains()
siads_csv = siads_csv[siads_csv['Unnamed: 0'].notna()]

# Aplicar o filtro somente nas linhas que contém '-'
# siads_csv = siads_csv[siads_csv['Unnamed: 0'].str.contains('-')]
print(siads_csv['MATERIAL:'].str.contains('-'))
siads_csv.to_clipboard(index=False)
# display(siads_csv)

11      NaN
12      NaN
13      NaN
18      NaN
23      NaN
       ... 
2897    NaN
2902    NaN
2907    NaN
2912    NaN
2917    NaN
Name: MATERIAL:, Length: 584, dtype: object


In [65]:
# print(siads_csv.iloc[0].str.contains(':') )
print(siads_csv.iloc[0].str.split(':') )

Unnamed: 0     [CONTA,  115610100 - MATERIAIS DE CONSUMO]
Unnamed: 12                                           NaN
Unnamed: 14                                           NaN
Name: 11, dtype: object


In [ ]:
siads_df = pd.DataFrame(columns=)

In [3]:
# Lendo todas as abas do arquivo Excel
file_path = '2021-10-17-SIADS_RelatorioInventario.xls'
siads_excel = pd.read_excel(file_path, sheet_name=None)

# Concatenando todas as abas em um único DataFrame
combined_df = pd.concat(siads_excel.values(), ignore_index=True)
combined_df = combined_df

# Exibindo as primeiras linhas do DataFrame combinado
print(combined_df.info())


C:\Users\Ricar\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10219 entries, 0 to 10218
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   0 non-null      float64
 1   Unnamed: 1   289 non-null    object 
 2   Unnamed: 2   86 non-null     object 
 3   Unnamed: 3   414 non-null    object 
 4   Unnamed: 4   0 non-null      float64
 5   Unnamed: 5   414 non-null    object 
 6   Unnamed: 6   414 non-null    object 
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   86 non-null     object 
 10  Unnamed: 10  414 non-null    object 
 11  Unnamed: 11  0 non-null      float64
 12  Unnamed: 12  0 non-null      float64
 13  Unnamed: 13  340 non-null    object 
 14  Unnamed: 14  4 non-null      object 
 15  Unnamed: 15  0 non-null      float64
 16  Unnamed: 16  0 non-null      float64
 17  Unnamed: 17  0 non-null      float64
 18  Unnamed: 18  0 non-null      float64
 19  Unna

In [4]:
combined_df.to_clipboard(index=False)